# Problem 1: ARMA(1,1) State Space Verification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/Quantlet_Problem01_ARMA_StateSpace.ipynb)

---

**QuantLet Name:** `SSM_ARMA_StateSpace_Verification`  
**Published in:** State Space Models and Markov Switching Models — Chapter 8  
**Author:** Daniel Traian Pele  
**Institution:** Bucharest University of Economic Studies  
**Date:** February 2026  

---

## Problem
Verify that the state space models (8.4) and (8.5) specify the ARMA(1,1) model (8.3).

**State space form:**
$$X_{t+1} = \varphi_0 + \varphi X_t + (\theta + \varphi)\varepsilon_t \quad (8.4)$$
$$Y_t = X_t + \varepsilon_t \quad (8.5)$$

**Target ARMA(1,1):**
$$Y_t = \varphi_0 + \varphi Y_{t-1} + \varepsilon_t + \theta\varepsilon_{t-1} \quad (8.3)$$

---
## Plotting: ✅ Transparent background · ✅ No grid · ✅ Legend outside bottom


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'axes.grid': False,
    'font.size': 11, 'axes.labelsize': 12,
    'axes.titlesize': 13, 'figure.figsize': (12, 5)
})
print('Setup complete.')


## 1. Algebraic Verification

From (8.5): $X_t = Y_t - \varepsilon_t$ and $X_{t-1} = Y_{t-1} - \varepsilon_{t-1}$.

Substituting into (8.4) shifted by one period:
$$X_t = \varphi_0 + \varphi X_{t-1} + (\theta + \varphi)\varepsilon_{t-1}$$
$$Y_t - \varepsilon_t = \varphi_0 + \varphi(Y_{t-1} - \varepsilon_{t-1}) + (\theta + \varphi)\varepsilon_{t-1}$$
$$Y_t = \varphi_0 + \varphi Y_{t-1} + \varepsilon_t + \theta\varepsilon_{t-1}$$

which is the ARMA(1,1) model (8.3). ∎


## 2. Numerical Simulation Verification


In [ ]:
# Set parameters
np.random.seed(42)
n = 1000
phi_0, phi, theta = 0.5, 0.6, 0.4
eps = np.random.normal(0, 1, n + 1)

# Method 1: State space simulation
X_ss = np.zeros(n + 1)
Y_ss = np.zeros(n + 1)
for t in range(n):
    X_ss[t + 1] = phi_0 + phi * X_ss[t] + (theta + phi) * eps[t]
    Y_ss[t] = X_ss[t] + eps[t]
Y_ss[n] = X_ss[n] + eps[n]
Y_state_space = Y_ss[1:]  # drop burn-in

# Method 2: Direct ARMA(1,1) simulation
Y_arma = np.zeros(n + 1)
for t in range(1, n + 1):
    Y_arma[t] = phi_0 + phi * Y_arma[t - 1] + eps[t] + theta * eps[t - 1]
Y_arma_direct = Y_arma[1:]

# Compare
max_diff = np.max(np.abs(Y_state_space - Y_arma_direct))
print(f'Maximum absolute difference: {max_diff:.2e}')
print(f'Are they equivalent? {"YES ✅" if max_diff < 1e-10 else "NO ❌"}')


In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 8))
fig.patch.set_alpha(0)

# Time series comparison (first 200 points)
ax = axes[0]
ax.patch.set_alpha(0); ax.grid(False)
ax.plot(Y_state_space[:200], color='steelblue', linewidth=1.0, label='State Space')
ax.plot(Y_arma_direct[:200], color='darkorange', linewidth=1.0, linestyle='--', label='ARMA(1,1)')
ax.set_title('State Space vs ARMA(1,1) — First 200 Observations',
             fontsize=13, fontweight='bold')
ax.set_ylabel('$Y_t$')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13), ncol=2, frameon=False)

# Difference
ax = axes[1]
ax.patch.set_alpha(0); ax.grid(False)
ax.plot(Y_state_space - Y_arma_direct, color='seagreen', linewidth=0.8)
ax.set_title('Difference (State Space − ARMA)', fontsize=13, fontweight='bold')
ax.set_ylabel('Difference')
ax.set_xlabel('Time')

plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.savefig('ARMA_StateSpace_Comparison.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()


In [ ]:
# Scatter plot
fig, ax = plt.subplots(figsize=(7, 7))
fig.patch.set_alpha(0); ax.patch.set_alpha(0); ax.grid(False)
ax.scatter(Y_state_space, Y_arma_direct, s=5, alpha=0.5, color='steelblue')
lims = [min(Y_state_space.min(), Y_arma_direct.min()),
        max(Y_state_space.max(), Y_arma_direct.max())]
ax.plot(lims, lims, 'r--', linewidth=1.0, label='45° line')
ax.set_xlabel('State Space $Y_t$')
ax.set_ylabel('ARMA(1,1) $Y_t$')
ax.set_title('Perfect Correspondence', fontsize=13, fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), ncol=1, frameon=False)
plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.savefig('ARMA_StateSpace_Scatter.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()


## Conclusion

Both algebraically and numerically, the state space representation (8.4)–(8.5) is exactly equivalent to the ARMA(1,1) model (8.3). The maximum numerical difference is at machine precision (~$10^{-15}$).
